In [ ]:
import pandas as pd
import heapq
import ipywidgets as widgets
from IPython.display import display, clear_output
import io
import itertools

In [ ]:
try:
    df = pd.read_csv('metro_cdmx_aristas.csv')
    df['from_stop_name'] = df['from_stop_name'].str.strip()
    df['to_stop_name'] = df['to_stop_name'].str.strip()
    df['route_long_name'] = df['route_long_name'].astype(str).str.strip()
except FileNotFoundError:
    print("No se encontró el archivo CSV.")

graph = {}

if 'df' in locals():
    for _, row in df.iterrows():
        u = row['from_stop_name']
        v = row['to_stop_name']
        weight = row['mean_travel_time_min']
        
        line_raw = str(row['route_short_name']).strip()
        line = line_raw.lstrip('L') 
        
        route_dir = row['route_long_name']

        if u not in graph: graph[u] = []
        graph[u].append((v, weight, line, route_dir))
        
    all_stations = sorted(list(graph.keys()))
else:
    all_stations = []

def calcular_ruta(origen, destino):
    counter = itertools.count()
    pq = [(0, next(counter), origen, None, [])]
    min_costs = {}
    
    while pq:
        current_time, _, current_node, current_line, path = heapq.heappop(pq)

        if current_node == destino:
            return current_time, path

        state = (current_node, current_line)
        if state in min_costs and min_costs[state] <= current_time:
            continue
        min_costs[state] = current_time

        if current_node in graph:
            for neighbor, travel_time, next_line, next_route_dir in graph[current_node]:
                new_time = current_time + travel_time
                penalty = 0
                
                line_str = f"L{next_line}"
                
                action = f"• {neighbor}"

                if current_line is not None and current_line != next_line:
                    penalty = 4
                    prev_line_str = f"L{current_line}"
                    
                    action = f"TRANSBORDO (Baja {prev_line_str}, sube {line_str}) en dirección {next_route_dir}"

                total_cost = new_time + penalty

                new_path = path + [{
                    'station': neighbor,
                    'line': next_line,
                    'action': action,
                    'direction': next_route_dir,
                    'time_leg': travel_time,
                    'penalty': penalty,
                    'is_transfer': (penalty > 0)
                }]

                heapq.heappush(pq, (total_cost, next(counter), neighbor, next_line, new_path))

    return float('inf'), []

In [ ]:
style = {'description_width': 'initial'}

w_origen = widgets.Dropdown(options=all_stations, description='Estación Origen:', style=style)
w_destino = widgets.Dropdown(options=all_stations, description='Estación Destino:', style=style)
w_boton = widgets.Button(description="Calcular Ruta", button_style='success')
w_output = widgets.Output()

def on_button_click(b):
    w_output.clear_output()
    with w_output:
        start = w_origen.value
        end = w_destino.value
        
        if start == end:
            print(" El origen y el destino son iguales.")
            return

        tiempo_total, pasos = calcular_ruta(start, end)
        
        if tiempo_total == float('inf'):
            print(f" No hay ruta disponible entre {start} y {end}")
        else:
            print(f" RUTA CALCULADA")
            print(f" Tiempo total estimado: {tiempo_total:.2f} minutos")
            print("-" * 60)
            print(f"INICIO: {start}")
            
            if pasos:
                primer_paso = pasos[0]
                linea = primer_paso['line']
                direccion = primer_paso['direction']
                print(f"Ingresa a la Línea {linea} en dirección {direccion}")
            
            for p in pasos:
                if p['is_transfer']:
                    print(f"{p['action']}")
                
                print(f"  • {p['station']} ({p['time_leg']:.1f} min)")
            
            print(f"LLEGADA: {end}")
w_boton.on_click(on_button_click)

display(widgets.VBox([
    widgets.HTML("<h2>Planeador de Rutas Metro CDMX</h2>"),
    widgets.HBox([w_origen, w_destino]),
    w_boton,
    w_output
]))
